# Planetary Nebula <a class="tocSkip">
    

In [3]:
import os
import sys

import errno
import warnings
import logging


from pathlib import Path

import numpy as np
import scipy as sp


%load_ext autoreload
%autoreload 2

import matplotlib as mpl
import matplotlib.pyplot as plt

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

logging.basicConfig(level=logging.INFO, stream=sys.stdout)

In [21]:
logging.basicConfig(filename='info.log',
                    filemode='a',
                    format='%(levelname)s %(name)s %(message)s',
                    datefmt='%H:%M:%S',
                    level=logging.DEBUG)

In [22]:
from pymuse.io import MUSEDAP

In [23]:
MUSEDAP('NGC628')

INFO:pymuse.io:loading NGC628


name: NGC628
filename: d:\downloads\MUSEDAP\NGC628\NGC628_MAPS.fits
lines
OIII5006
header
wcs
shape
OIII5006_err
HA6562
HA6562_err
NII6583
NII6583_err
SII6716
SII6716_err
PSF

## Reference files

In [6]:
from astropy.io import ascii
from astropy.coordinates import SkyCoord              # convert pixel to sky coordinates

In [7]:
pn_kreckel = ascii.read(os.path.join('..','data','external','kreckel_pn_2017.txt'))

def string_to_ra(string):
    '''convert coordinates from Kreckel et al. (2017) to astropy
    
    the right ascension in the paper is given as 
    "01:36:42.212" but astropy requires "01h36m42.212s".
    This function replaces the ":" with the appropriate character.
    '''
    return string.replace(':','h',1).replace(':','m') + 's'

def string_to_dec(string):
    '''convert coordinates from Kreckel et al. (2017) to astropy
    
    the declination in the paper is given as "01:36:42.212" 
    but astropy requires "01d36m42.212s".
    This function replaces the ":" with the appropriate character.
    '''
    return string.replace(':','d',1).replace(':','m') + 's'

# convert string to astronomical coordinates
pn_kreckel['RA'] = list(map(string_to_ra,pn_kreckel['RA']))
pn_kreckel['DEC'] = list(map(string_to_dec,pn_kreckel['DEC']))
pn_kreckel['SkyCoord'] = SkyCoord(pn_kreckel['RA'],pn_kreckel['DEC'])

# select some subsets (PN from Hermann et al. 2008 or bright sources only)
pn_herrmann = pn_kreckel[[True if i.endswith('a') else False for i in pn_kreckel['ID']]]
pn_bright = pn_kreckel[pn_kreckel['mOIII']<27]

## Read in data

In [8]:
from pymuse.io import MUSEDAP